<center>
<img src="http://corpuslg.org/lael_english/wp-content/uploads/2020/04/lael_50_years_narrow_white.png.400px_300dpi.png" width="300" alt="LAEL 50 years logo">
<h3>APPLIED LINGUISTICS GRADUATE PROGRAMME (LAEL)</h3>
</center>
<hr>

# Images function set

## Preamble

## presample

## collecturls

## grabimages

## removedupes

## uploadtobucket

## googlelabels

## labeltypes

## toplabels

## sas

## datamatrix

## correlationmatrux

## formats

## examples